## Тематическая модель на данных Last.fm

Сначала настраиваем и подключаем нужные библиотеки

In [1]:
import os
import sys
HOME = '/home/vovapolu/Projects/'
BIGARTM_PATH = HOME + 'bigartm/'
BIGARTM_BUILD_PATH = BIGARTM_PATH + 'build/'
sys.path.append(os.path.join(BIGARTM_PATH, 'src/python'))
os.environ['ARTM_SHARED_LIBRARY'] = os.path.join(BIGARTM_BUILD_PATH, 'src/artm/libartm.so')

Теперь создаем батч и map'ы для исполнителей

In [2]:
import artm.messages_pb2, artm.library, uuid
import csv

plays_file = 'usersha1-artmbid-artname-plays.tsv'

batch = artm.messages_pb2.Batch()
batch.id = str(uuid.uuid4())
artist_id_to_name = {}
artists_idxs = {}
artists = []

Читаем файл и строим батч

In [ ]:
last_user_id = ''
now_item_id = 0

users_to_handle = 5000

with open(plays_file, 'rb') as tsvin:
    tsvin = csv.reader(tsvin, delimiter='\t')

    field = None

    for row in tsvin:

        if now_item_id > users_to_handle:
            break

        user_id, artist_id, artist_name, plays = row
        if artist_id not in artists:
            artist_id_to_name[artist_id] = artist_name
            artists_idxs[artist_id] = len(artists)
            artists.append(artist_name)

        if user_id != last_user_id:
            item = batch.item.add()
            item.id = now_item_id
            now_item_id += 1

            field = item.field.add()

            last_user_id = user_id

        field.token_id.append(artists_idxs[artist_id])
        field.token_count.append(int(plays))

for artist in artists:
    try:
        ascii_artist = artist.decode('ascii')
    except UnicodeDecodeError:
        print artist
        print "it was not a ascii-encoded unicode string"
        print "added empty string as artist name"
        batch.token.append('')
    else:
        batch.token.append(ascii_artist)

Запускаем BigArtm

In [4]:
with artm.library.MasterComponent() as master:
    perplexity_score = master.CreatePerplexityScore()
    top_tokens_score = master.CreateTopTokensScore(num_tokens=4)
    model = master.CreateModel(topics_count=100, inner_iterations_count=10)

    for iteration in range(0, 50):
        master.AddBatch(batch)
        master.WaitIdle()
        model.Synchronize()
        print "Iteration#" + str(iteration) + ": Perplexity = %.3f" % perplexity_score.GetValue(model).value

    top_tokens = top_tokens_score.GetValue(model)

artm.library.Visualizers.PrintTopTokensScore(top_tokens)

Iteration#0: Perplexity = 0.000
Iteration#1: Perplexity = 13289.567
Iteration#2: Perplexity = 3592.548
Iteration#3: Perplexity = 1486.924
Iteration#4: Perplexity = 870.014
Iteration#5: Perplexity = 617.223
Iteration#6: Perplexity = 487.607
Iteration#7: Perplexity = 410.929
Iteration#8: Perplexity = 360.966
Iteration#9: Perplexity = 326.116
Iteration#10: Perplexity = 300.567
Iteration#11: Perplexity = 281.101
Iteration#12: Perplexity = 265.806
Iteration#13: Perplexity = 253.489
Iteration#14: Perplexity = 243.365
Iteration#15: Perplexity = 234.900
Iteration#16: Perplexity = 227.720
Iteration#17: Perplexity = 221.554
Iteration#18: Perplexity = 216.202
Iteration#19: Perplexity = 211.510
Top tokens per topic: 
Topic#1:  lake of tears(0.025)  metallica(0.022)  spoon(0.022)  kreator(0.022)  
Topic#2:  (0.246)  mitsumune shinkichi(0.077)  nagaoka seikou(0.056)  key(0.037)  
Topic#3:  the orb(0.075)  (0.027)  coma(0.026)  ulrich schnauss(0.023)  
Topic#4:  avenged sevenfold(0.044)  system of a 

libartm.so: cannot open shared object file: No such file or directory, fall back to ARTM_SHARED_LIBRARY environment variable
